In [1]:
import cv2
import numpy as np

In [2]:
import cv2
import numpy as np

# Define a range for the color (e.g., blue in HSV format)
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

# Initialize a blank canvas
canvas = None

# Initialize variables to store previous position for drawing
prev_x, prev_y = None, None

# Access the webcam
cap = cv2.VideoCapture(0)

while True:
    # Read each frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally (optional)
    frame = cv2.flip(frame, 1)

    # Create a blank canvas with the same dimensions as the frame if not already created
    if canvas is None:
        canvas = np.zeros_like(frame)

    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a binary mask where the color falls within the range
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the largest contour by area
        max_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(max_contour) > 1000:
            # Get the center of the contour
            M = cv2.moments(max_contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])

                # If we have previous points, draw a line
                if prev_x is not None and prev_y is not None:
                    cv2.line(canvas, (prev_x, prev_y), (cx, cy), (255, 0, 0), 5)

                # Update previous points
                prev_x, prev_y = cx, cy
            else:
                prev_x, prev_y = None, None
        else:
            prev_x, prev_y = None, None
    else:
        prev_x, prev_y = None, None

    # Combine the canvas and the original frame
    output = cv2.add(frame, canvas)

    # Display the result
    cv2.imshow("Air Canvas", output)
    cv2.imshow("Mask", mask)

    # Exit when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
import mediapipe as mp

In [ ]:
# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

# Initialize canvas and drawing variables
canvas = None
prev_x, prev_y = None, None
drawing = False  # Flag to indicate if we should draw

# Access the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip frame horizontally
    h, w, _ = frame.shape

    # Create a blank canvas if it doesn't exist
    if canvas is None:
        canvas = np.zeros((h, w, 3), dtype=np.uint8)

    # Convert the frame to RGB for MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Get landmarks for index finger and thumb
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

            # Convert normalized coordinates to pixel values
            ix, iy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
            tx, ty = int(thumb_tip.x * w), int(thumb_tip.y * h)

            # Calculate the distance between index and thumb to detect pinch
            distance = np.hypot(tx - ix, ty - iy)

            # Drawing Mode (if index finger up and not pinching)
            if distance > 40:  # Adjust threshold as needed
                drawing = True
                if prev_x is not None and prev_y is not None:
                    cv2.line(canvas, (prev_x, prev_y), (ix, iy), (255, 0, 0), 5)
                prev_x, prev_y = ix, iy
            else:
                drawing = False
                prev_x, prev_y = None, None  # Reset previous points to stop drawing

            # Eraser Mode (if pinching)
            if 0 < distance < 40:
                cv2.circle(canvas, (ix, iy), 20, (0, 0, 0), -1)  # Draw a black circle to erase

            # Draw hand landmarks on the frame (optional)
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    else:
        drawing = False
        prev_x, prev_y = None, None

    # Combine canvas and the frame
    output = cv2.add(frame, canvas)

    # Display the output
    cv2.imshow("Air Canvas - Hand Gesture", output)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
hands.close()


KeyboardInterrupt: 

: 

In [2]:
# All the imports go here
import cv2
import numpy as np
import mediapipe as mp
from collections import deque


# Giving different arrays to handle colour points of different colour
bpoints = [deque(maxlen=1024)]
gpoints = [deque(maxlen=1024)]
rpoints = [deque(maxlen=1024)]
ypoints = [deque(maxlen=1024)]


# These indexes will be used to mark the points in particular arrays of specific colour
blue_index = 0
green_index = 0
red_index = 0
yellow_index = 0

#The kernel to be used for dilation purpose 
kernel = np.ones((5,5),np.uint8)

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
colorIndex = 0

# Here is code for Canvas setup
paintWindow = np.zeros((1920,1080,3)) + 255
paintWindow = cv2.rectangle(paintWindow, (40,1), (140,65), (0,0,0), 2)
paintWindow = cv2.rectangle(paintWindow, (160,1), (255,65), (255,0,0), 2)
paintWindow = cv2.rectangle(paintWindow, (275,1), (370,65), (0,255,0), 2)
paintWindow = cv2.rectangle(paintWindow, (390,1), (485,65), (0,0,255), 2)
paintWindow = cv2.rectangle(paintWindow, (505,1), (600,65), (0,255,255), 2)

cv2.putText(paintWindow, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)


# initialize mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.9)
mpDraw = mp.solutions.drawing_utils


# Initialize the webcam
cap = cv2.VideoCapture(0)
ret = True
while ret:
    # Read each frame from the webcam
    ret, frame = cap.read()

    x, y, c = frame.shape

    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    #hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    frame = cv2.rectangle(frame, (40,1), (140,65), (0,0,0), 2)
    frame = cv2.rectangle(frame, (160,1), (255,65), (255,0,0), 2)
    frame = cv2.rectangle(frame, (275,1), (370,65), (0,255,0), 2)
    frame = cv2.rectangle(frame, (390,1), (485,65), (0,0,255), 2)
    frame = cv2.rectangle(frame, (505,1), (600,65), (0,255,255), 2)
    cv2.putText(frame, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    #frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # Get hand landmark prediction
    result = hands.process(framergb)

    # post process the result
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                # # print(id, lm)
                # print(lm.x)
                # print(lm.y)
                lmx = int(lm.x * 640)
                lmy = int(lm.y * 480)

                landmarks.append([lmx, lmy])


            # Drawing landmarks on frames
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)
        fore_finger = (landmarks[8][0],landmarks[8][1])
        center = fore_finger
        thumb = (landmarks[4][0],landmarks[4][1])
        cv2.circle(frame, center, 3, (0,255,0),-1)
        print(center[1]-thumb[1])
        if (thumb[1]-center[1]<30):
            bpoints.append(deque(maxlen=512))
            blue_index += 1
            gpoints.append(deque(maxlen=512))
            green_index += 1
            rpoints.append(deque(maxlen=512))
            red_index += 1
            ypoints.append(deque(maxlen=512))
            yellow_index += 1

        elif center[1] <= 65:
            if 40 <= center[0] <= 140: # Clear Button
                bpoints = [deque(maxlen=512)]
                gpoints = [deque(maxlen=512)]
                rpoints = [deque(maxlen=512)]
                ypoints = [deque(maxlen=512)]

                blue_index = 0
                green_index = 0
                red_index = 0
                yellow_index = 0

                paintWindow[67:,:,:] = 255
            elif 160 <= center[0] <= 255:
                    colorIndex = 0 # Blue
            elif 275 <= center[0] <= 370:
                    colorIndex = 1 # Green
            elif 390 <= center[0] <= 485:
                    colorIndex = 2 # Red
            elif 505 <= center[0] <= 600:
                    colorIndex = 3 # Yellow
        else :
            if colorIndex == 0:
                bpoints[blue_index].appendleft(center)
            elif colorIndex == 1:
                gpoints[green_index].appendleft(center)
            elif colorIndex == 2:
                rpoints[red_index].appendleft(center)
            elif colorIndex == 3:
                ypoints[yellow_index].appendleft(center)
    # Append the next deques when nothing is detected to avois messing up
    else:
        bpoints.append(deque(maxlen=512))
        blue_index += 1
        gpoints.append(deque(maxlen=512))
        green_index += 1
        rpoints.append(deque(maxlen=512))
        red_index += 1
        ypoints.append(deque(maxlen=512))
        yellow_index += 1

    # Draw lines of all the colors on the canvas and frame
    points = [bpoints, gpoints, rpoints, ypoints]
    # for j in range(len(points[0])):
    #         for k in range(1, len(points[0][j])):
    #             if points[0][j][k - 1] is None or points[0][j][k] is None:
    #                 continue
    #             cv2.line(paintWindow, points[0][j][k - 1], points[0][j][k], colors[0], 2)
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    cv2.imshow("Output", frame) 
    cv2.imshow("Paint", paintWindow)

    if cv2.waitKey(1) == ord('q'):
        break;

    # if cv2.waitkey(1) & 0xFF == ord('s'):
    #      cv2.imwrite("canvas_output.png", paintWindow)

# release the webcam and destroy all active windows
cap.release()
cv2.destroyAllWindows()

-1
-30
-27
-28
-30
-32
-29
-29
-28
-26
-29
-25
-30
-30
-29
-26
-27
-22
-21
-25
-26
-21
-24
-21
-20
-22
-22
-20
-19
-22
-19
-20
-23
-25
-20
-27
-28
-30
-26
-25
-32
-28
-30
-28
-26
-25
-26
-25
-25
-25
-26
-26
-28
-23
-16
-21
-28
-42
-49
-56
-60
-71
-74
-73
-75
-70
-49
-54
-86
-89
-96
-98
-95
-94
-94
-97
-96
-96
-95
-93
-93
-90
-90
-83
-79
-79
-80
-74
-77
-79
-81
-83
-78
-22
-71
-78
-76
-75
-72
-67
-70
-67
-65
-66
-68
-67
-70
-71
-74
-73
-72
-72
-72
-70
-72
-73
-74
-70
-71
-72
-69
-66
-63
-63
-61
-64
-63
-62
-65
-68
-68
-68
-68
-72
-70
-72
-71
-72
-71
-66
-52
-56
-60
-61
-60
-59
-52
-51
-54
-56
-56
-59
-56
-56
-61
-62
-56
-66
-66
-64
-65
-65
-62
-59
-56
-39
-77
-44
-38
-44
-40
-41
-40
-31
-42
-39
-45
-47
-40
-40
-44
-48
-16
-82
-71
-62
-65
-69
-65
-64
-46
-65
-67
-61
-60
-55
-57
-51
-48
-50
-46
-45
-43
-42
-34
-19
-140
-140
-139
-141
-139
-138
-139
-140
-142
-137
-79
-85
-86
-92
-93
-93
-91
-91
-90
-89
-90
-91
-91
-91
-92
-91
-91
-89
-91
-91
-89
-91
-91
-88
-89
-88
-87
-86
-85
-83
-7
-7
-

In [1]:
import cv2
import numpy as np
import mediapipe as mp
from collections import deque

# Initialize deque to store signature points
signature_points = [deque(maxlen=1024)]
point_index = 0

# Setup kernel for dilation (if needed for thickness)
kernel = np.ones((5, 5), np.uint8)

# Setup canvas for signature
canvas = np.zeros((471, 636, 3), dtype=np.uint8) + 255  # White background

# Initialize MediaPipe Hands
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame from webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Flip frame for natural mirroring
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process hand landmarks
    result = hands.process(framergb)

    # Check if hand is detected
    if result.multi_hand_landmarks:
        landmarks = []
        for handLms in result.multi_hand_landmarks:
            for lm in handLms.landmark:
                lmx, lmy = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                landmarks.append([lmx, lmy])

            # Draw landmarks on frame for visual feedback (optional)
            mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS)

        # Get position of index finger (landmark 8) and thumb (landmark 4)
        index_finger = (landmarks[8][0], landmarks[8][1])
        thumb = (landmarks[4][0], landmarks[4][1])

        # Capture signature only if thumb and index finger are close (indicating writing mode)
        if abs(index_finger[1] - thumb[1]) < 30:
            signature_points.append(deque(maxlen=512))
            point_index += 1
        else:
            # Append points for the signature path
            signature_points[point_index].appendleft(index_finger)

    else:
        # Start new deque when hand is not detected to avoid jumbling points
        signature_points.append(deque(maxlen=512))
        point_index += 1

    # Draw signature lines on both the frame and canvas
    for j in range(len(signature_points)):
        for k in range(1, len(signature_points[j])):
            if signature_points[j][k - 1] is None or signature_points[j][k] is None:
                continue
            cv2.line(frame, signature_points[j][k - 1], signature_points[j][k], (0, 0, 0), 2)
            cv2.line(canvas, signature_points[j][k - 1], signature_points[j][k], (0, 0, 0), 2)

    # Display the frame and canvas
    cv2.imshow("Live Feed", frame)
    cv2.imshow("Signature Capture", canvas)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 